In [215]:
import math
from ast import literal_eval

import plotly.express as px
import plotly.subplots as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
from matrepr import mdisplay
from plotly.subplots import make_subplots
from rich import print as rprint
from safetensors.torch import load_file, load_model
from torch import linalg
from torch.nn.utils.parametrizations import orthogonal
from torch.utils.data import DataLoader, Dataset
from torcheval.metrics import MulticlassAccuracy
from torchvision import transforms

from koopmann import aesthetics
from koopmann.data import (
    DatasetConfig,
    create_data_loader,
    get_dataset_class,
)
from koopmann.models import (
    MLP,
    Autoencoder,
    ExponentialKoopmanAutencoder,
    LowRankKoopmanAutoencoder,
)
from koopmann.models.utils import (
    get_device,
    pad_act,
    parse_safetensors_metadata,
)

# from koopmann.utils import compute_model_accuracy
from koopmann.visualization import plot_eigenvalues

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
task = "mnist"
scale_idx = "0"
k = "1"
dim = "1024"
flavor = "lowrank_20"
# flavor = "standard"
# flavor = "exponential"
user = "nsa325"


In [217]:
model_name = f"{task}_probed"
ae_name = f"{task}_model"

# Original model path
model_file_path = f"/scratch/{user}/koopmann_model_saves/{model_name}.safetensors"

# Autoenoder path in work dir
ae_file_path = f"/scratch/{user}/koopmann_model_saves/scaling/dim_{dim}_k_{k}_loc_{scale_idx}_{flavor}_autoencoder_{ae_name}.safetensors"

In [218]:
model, _ = MLP.load_model(model_file_path)
model.modules[-2].remove_nonlinearity()
# model.modules[-3].update_nonlinearity("leakyrelu")
model.modules[-3].remove_nonlinearity()
model.eval()
model.hook_model()

In [219]:
metadata = parse_safetensors_metadata(file_path=model_file_path)

dataset_config = DatasetConfig(
    dataset_name=metadata["dataset"],
    num_samples=5_000,
    split="test",
    seed=21,
)
DatasetClass = get_dataset_class(name=dataset_config.dataset_name)
dataset = DatasetClass(config=dataset_config)
dataloader = create_data_loader(dataset, batch_size=1024)

In [220]:
# Parse metadata
metadata = parse_safetensors_metadata(file_path=ae_file_path)

# Choose model based on flag
if "standard" in flavor:
    AutoencoderClass = Autoencoder
elif "lowrank" in flavor:
    AutoencoderClass = LowRankKoopmanAutoencoder
elif "exponential" in flavor:
    AutoencoderClass = ExponentialKoopmanAutencoder

# Instantiate model
autoencoder = AutoencoderClass(
    input_dimension=literal_eval(metadata["input_dimension"]),
    latent_dimension=literal_eval(metadata["latent_dimension"]),
    nonlinearity=metadata["nonlinearity"],
    k=literal_eval(metadata["steps"]),
    batchnorm=literal_eval(metadata["batchnorm"]),
    hidden_configuration=literal_eval(metadata["hidden_configuration"]),
    rank=literal_eval(metadata["rank"]),
)

# Load weights
load_model(autoencoder, ae_file_path, device="cpu", strict=True)
autoencoder.eval()

K_matrix = autoencoder.koopman_matrix.linear_layer.weight.T.detach()

k = literal_eval(metadata["num_scaled"])
print(f"Little K: {k}")

Little K: 1


In [221]:
images, labels = next(iter(dataloader))

In [222]:
sample_idx = torch.randint(1024, (1,))[0].item()

In [223]:
with torch.no_grad():
    x = images.flatten(start_dim=1)
    # TODO: This is quick and dirty
    ###############################################################
    # Undo MNIST standardization: X_original = X_standardized * std + mean
    x = x * 0.3081 + 0.1307

    # Convert [0,1] range to [-1,1]
    x = 2 * x - 1
    ###############################################################
    x_obs = autoencoder._encode(x)
    pred_obs = x_obs @ torch.linalg.matrix_power(K_matrix, k)
    x_recon = autoencoder._decode(x_obs)
    x_pred = autoencoder._decode(pred_obs)

In [224]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=["Original MLP Input", "Reconstructed MLP Input"]
)

for i, img in enumerate([x[sample_idx], x_recon[sample_idx]], 1):
    fig.add_trace(px.imshow(img.reshape(28, 28)).data[0], row=1, col=i)

fig.update_layout(height=400, width=800, xaxis_scaleanchor="y", xaxis2_scaleanchor="y2")
fig.show()

print(f"Error: {F.mse_loss(x[sample_idx], x_recon[sample_idx], reduction='mean')}")

Error: 0.018307074904441833


In [225]:
with torch.no_grad():
    x = images.flatten(start_dim=1)
    _ = model(x)
    # TODO: This is quick and dirty
    ###############################################################
    # Undo MNIST standardization: X_original = X_standardized * std + mean
    x = x * 0.3081 + 0.1307

    # Convert [0,1] range to [-1,1]
    x = 2 * x - 1
    ###############################################################
    act_dict = model.get_fwd_activations()
    probe_act = act_dict[4]
    act_obs = autoencoder._encode(probe_act)
    probe_recon = autoencoder._decode(act_obs)

In [226]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=["Original MLP Activation", "Reconstructed MLP Activation"]
)

for i, img in enumerate([probe_act[sample_idx], probe_recon[sample_idx]], 1):
    fig.add_trace(px.imshow(img.reshape(28, 28)).data[0], row=1, col=i)

fig.update_layout(height=400, width=800, xaxis_scaleanchor="y", xaxis2_scaleanchor="y2")
fig.show()
print(
    f"Reconstruction Error: {F.mse_loss(probe_act[sample_idx], probe_recon[sample_idx], reduction='mean')}"
)

Reconstruction Error: 0.0001563559635542333


In [227]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        f"Input Label {labels[sample_idx]} in Observable Space",
        f"Activation {labels[sample_idx]} in Observable Space",
    ],
)

for i, img in enumerate([x_obs[sample_idx], act_obs[sample_idx]], 1):
    fig.add_trace(
        px.imshow(img.reshape(32, 32)).data[0],
        row=1,
        col=i,
    )

fig.update_layout(height=400, width=800, xaxis_scaleanchor="y", xaxis2_scaleanchor="y2")
fig.show()

rprint(f"Min Input Observable: {torch.topk(x_obs[sample_idx], k=1, largest=False).values.item()}")
rprint(f"Max Input Observable: {torch.topk(x_obs[sample_idx], k=1).values.item()}")

rprint(
    f"Min Activation Observable: {torch.topk(act_obs[sample_idx], k=1, largest=False).values.item()}"
)
rprint(f"Max Activation Observable: {torch.topk(act_obs[sample_idx], k=1).values.item()}")

Min Input Observable: -0.06569058448076248

Max Input Observable: 6.505367755889893

Min Activation Observable: -0.014206492342054844

Max Activation Observable: 2.2879467010498047

In [228]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=["Activation Observable ", "Predicted Activation Observable"]
)

for i, img in enumerate([act_obs[sample_idx], pred_obs[sample_idx]], 1):
    fig.add_trace(
        px.imshow(img.reshape(32, 32)).data[0],
        row=1,
        col=i,
    )

fig.update_layout(height=400, width=800, xaxis_scaleanchor="y", xaxis2_scaleanchor="y2")
fig.show()
print(
    f"Linear Prediction Error: {F.mse_loss(act_obs[sample_idx], pred_obs[sample_idx], reduction='mean')}"
)

Linear Prediction Error: 0.0008903853595256805


In [229]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=["Original MLP Activation", "Predicted MLP Activation"]
)

for i, img in enumerate([probe_act[sample_idx], x_pred[sample_idx]], 1):
    fig.add_trace(px.imshow(img.reshape(28, 28)).data[0], row=1, col=i)

fig.update_layout(height=400, width=800, xaxis_scaleanchor="y", xaxis2_scaleanchor="y2")
fig.show()
print(f"Error: {F.mse_loss(probe_act[sample_idx], x_pred[sample_idx], reduction='mean')}")

Error: 0.009512532502412796


In [230]:
def compute_model_accuracy(model, dataset, batch_size=1_024):
    model.eval()
    model.hook_model()
    dataloader = DataLoader(dataset, batch_size=batch_size)

    metric_original = MulticlassAccuracy()
    metric_altered = MulticlassAccuracy()

    device = get_device()

    for batch in dataloader:
        input, target = batch
        input, target = input.to(device), target.to(device)

        output_original = model(input)

        # TODO: This is quick and dirty
        ###############################################################
        # Undo MNIST standardization: X_original = X_standardized * std + mean
        input = input * 0.3081 + 0.1307

        # Convert [0,1] range to [-1,1]
        input = 2 * input - 1
        ###############################################################

        pred_act = autoencoder(input.flatten(start_dim=1), k=k).predictions[-1]
        output_altered = model.modules[-2:](pred_act)

        metric_original.update(output_original, target.squeeze())
        metric_altered.update(output_altered, target.squeeze())
    return metric_original.compute(), metric_altered.compute()


acc_orig, acc_ae = compute_model_accuracy(model.to("cpu"), dataset)
print(f"Original Model Testing Accuracy: {acc_orig}")
print(f"Autoencoder Prediction Testing Accuracy: {acc_ae}")

Original Model Testing Accuracy: 0.9771000146865845
Autoencoder Prediction Testing Accuracy: 0.9753000140190125
